## KNN ##

k개의 가까운 이웃이라는 뜻, 새로운 관측값이 주어지면 기존 데이터 중에서 가장 속성이 비슷한 k개의 이웃을 먼저 찾는다. 그리고 가까운 이웃들이 갖고 있는 목표 값과 같은 값으로 분류하여 예측한다. (이웃과의 유사성 측정하기 위한 수학적 개념 공부 필요)

k값에 따라 예측의 정확도가 달라지므로 적절한 k값에 따라서 예측 결과인 분류값이 달라진다(관측값에 가까운 k개의 값중 가장 개수가 많은 값으로 분류)

## 타이타닉 생존자 예측 ##

In [1]:
import pandas as pd
import seaborn as sns


In [2]:
# seaborn에서 제공하는 titanic 데이터셋 가져오기

# load_dataset이라는 함수를 사용하여 데이터프레임으로 변환
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
# 출력할 열의 개수 한도 늘리기

pd.set_option('display.max_columns', 15)
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [4]:
# 데이터 탐색
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [5]:
# deck은 누락된 값이 688개이므로 열 제거하고 embark_town은 embark와 동일하므로 중복 제거 
rdf = df.drop(['deck', 'embark_town'], axis=1)
rdf.columns.values

array(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'alive', 'alone'],
      dtype=object)

In [6]:
# age열 누락 데이터 177개 , 분석에 포함하는 중요한 속성이므로 평균 나이로 치환하거나 누락 데이터가 있는 행 모두 제거->우선 714명 데이터만 분석
# age열에 데이터가 없는 모든 행 삭제
rdf = rdf.dropna(subset=['age'], how='any', axis=0)
len(rdf)

714

In [7]:
# embarked 열엔 승객들이 타이타닉호에 탑승한 도시명의 첫글자있음, 누락 데이터는 2개이므로 탑승한 승객이 가장 많은 도시명으로 치환
# 가장 많이 탑승한 도시명 알아내기 방법1

most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
most_freq


'S'

In [8]:
# 방법2 - embarked 열의 최빈값 top 확인하면 s

rdf.describe(include='all')

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
count,714.000000,714.000000,714,714.000000,714.000000,714.000000,714.000000,712,714,714,714,714,714
unique,NaN,NaN,2,NaN,NaN,NaN,NaN,3,3,3,2,2,2
top,NaN,NaN,male,NaN,NaN,NaN,NaN,S,Third,man,True,no,True
freq,NaN,NaN,453,NaN,NaN,NaN,NaN,554,355,413,413,424,404
mean,0.406162,2.236695,NaN,29.699118,0.512605,0.431373,34.694514,NaN,NaN,NaN,NaN,NaN,NaN
std,0.491460,0.838250,NaN,14.526497,0.929783,0.853289,52.918930,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,1.000000,NaN,0.420000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,1.000000,NaN,20.125000,0.000000,0.000000,8.050000,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,2.000000,NaN,28.000000,0.000000,0.000000,15.741700,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.000000,3.000000,NaN,38.000000,1.000000,1.000000,33.375000,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# 누락데이터 변환하기
rdf['embarked'].fillna(most_freq, inplace=True)

In [10]:
rdf

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,29.1250,Q,Third,woman,False,no,False
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,yes,True
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,yes,True


In [13]:
# 속성 선택
ndf = rdf[['survived', 'pclass','sex','age','sibsp','parch','embarked']]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [14]:
# sex, embarked범주형 데이터 숫자형으로 변환하는 더미 변수 만들기(원 핫 인코딩)
onehot_sex = pd.get_dummies(ndf['sex'])#sex열 더미변수화
ndf = pd.concat([ndf, onehot_sex], axis=1)#기존 데이프레임 연결


In [15]:
onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')#열 이름에 접두어 town 붙이고 더미변수화
ndf = pd.concat([ndf, onehot_embarked], axis=1)#기존 데이터프레임 연결

In [18]:
#기존 열 삭제
ndf.drop(['sex','embarked'], axis=1, inplace=True)
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


In [22]:
# 훈련/검증 데이터 분할

# 종속변수인 survived는 지도학습이므로 미리 열을 만들어두어야 함, 
# survived를 예측 변수인 y에 저장하고 나머지 열들을 설명변수로 X에 할당

X = ndf[['pclass','age','sibsp','parch','female','male','town_C','town_Q','town_S']]#설명변수 X
y = ndf['survived']#예측변수 Y

In [23]:
# 설명 변수 열들이 갖는 데이터의 상대적 크기 차이를 없애기 위해 정규화 과정(normalization) 거친다

from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

In [26]:
# train data와 test data로 구분(7:3 비율)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

print('train data 개수:', X_train.shape)

print('test data 개수:', X_test.shape)

train data 개수: (499, 9)
test data 개수: (215, 9)


In [29]:
# KNN분류모형 사용

from sklearn.neighbors import KNeighborsClassifier

# 모형객체 생성(k=5로 설정)
knn = KNeighborsClassifier(n_neighbors=5)

In [30]:
# train data를 가지고 모형 학습
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [31]:
# test data를 가지고 y_hat을 예측(분류)
y_hat = knn.predict(X_test)

In [35]:
print(y_hat[0:10])
print(y_test.values[0:10])

[0 0 1 0 0 1 1 1 0 0]
[0 0 1 0 0 1 1 1 0 0]


In [38]:
# 모형의 성능(예측 능력)을 평가 confusion matrix 계산

from sklearn import metrics
knn_matrix = metrics.confusion_matrix(y_test, y_hat)
print(knn_matrix)

# TP, FP
# FN, TN

# 총 4개 값을 더하면 215명 중에서 미생존자를 정확히 예측한 수는 109명, 미생존자를 생존자로 잘못 분류한 FP는 16명
# 생존자를 미생존자로 잘못 분류한 FN은 25명, 생존자를 정확하게 예측한 TN은 65명

# 정확도(precision) = TP/TP+FP 
# 109/125 = 0.872 (False를 True로 잘못 예측한 오류가 작다)
# 재현율(recall) = TP/TP+FN
# 109/134 = 0.813 (모형의 안정성을 나타내는 지표, 실제 True를 False로 잘못 예측한 오류가 낮다)


[[109  16]
 [ 25  65]]


In [41]:
# 모형의 성능 평가 - F1스코어 평가 지표 계산
knn_report = metrics.classification_report(y_test, y_hat)
print(knn_report,'\n')

# f1-score를 보면 미생존자(0) 예측 정확도가 0.84, 생존자 예측(1)이 0.76으로 예측 능력에 차이가 있음

              precision    recall  f1-score   support

           0       0.81      0.87      0.84       125
           1       0.80      0.72      0.76        90

    accuracy                           0.81       215
   macro avg       0.81      0.80      0.80       215
weighted avg       0.81      0.81      0.81       215
 



In [ ]:
# 더 공부해볼 것, K개를 5개로 설정하였는데 일반적으로 몇개를 설정해야 하는지? 기준은?

# K의 default 값은 5입니다. 가장 가까운 주변 5개 데이터를 기반으로 분류한다는 것입니다. 일반적으로 K는 홀수를 사용합니다. 짝수일 경우 동점이 되어 하나의 결과를 도출할 수 없기 때문입니다.

# 로지스틱 회귀분석으로 분류할 시 예측 정확도는 높을까?

## 로지스틱 회귀분석 ##

In [43]:
!pip install statsmodels

In [49]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import statsmodels.api as sm

In [62]:
model = sm.Logit(y_train, X_train)#종속변수, 독립변수 train을 로짓화 
result = model.fit()

         Current function value: 0.464932
         Iterations: 35


C:\ProgramData\Anaconda3\envs\pydata\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


In [63]:
result.summary()
# 각 첫번째~9번째 변수 회귀계수, 표준 오자, z 스코어 등 뜻은?
# z값 계수와 계수의 표준 오차 간의 비율을 측정, 0에서 충분히 떨어져 있는 Z-값은 계수 추정치가 0과 통계적으로 다를만큼 충분히 크고 정확함을 나타냅니다. 반대로, 0과 가까운 Z-값은 항이 반응에 영향을 미친다고 확신하기에는 계수가 너무 작거나 너무 부정확함을 나타냅니다. .
# p-값은 귀무 가설에 반하는 증거를 측정하는 확률입니다. p-값이 작을수록 귀무 가설에 반하는 더 강력한 증거가 됩니다.
# 5번째 변수가 0.6447로 생존 예측에 가장 유의한 변수임

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               survived   No. Observations:                  499
Model:                          Logit   Df Residuals:                      492
Method:                           MLE   Df Model:                            6
Date:                Mon, 17 May 2021   Pseudo R-squ.:                  0.3095
Time:                        20:43:49   Log-Likelihood:                -232.00
converged:                      False   LL-Null:                       -335.99
Covariance Type:            nonrobust   LLR p-value:                 3.781e-42
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -1.0397      0.142     -7.337      0.000      -1.317      -0.762
x2            -0.5692      0.137     -4.143      0.000      -0.838      -0.300
x3            -0.3001      0.140     -2.144      0.032      -0.574      -0.026
x4            -0.0575      0.129     -0.446      0.656      -0.311       0.195
x5             0.6447        nan        nan        nan         nan         nan
x6            -0.6447        nan        nan        nan         nan         nan
x7             0.1237   3.37e+06   3.67e-08      1.000    -6.6e+06     6.6e+06
x8            -0.0318   1.69e+06  -1.88e-08      1.000   -3.32e+06    3.32e+06
x9            -0.1001   3.62e+06  -2.76e-08      1.000    -7.1e+06     7.1e+06
==============================================================================
"""

In [47]:
results.params

x1   -1.039688
x2   -0.569183
x3   -0.300089
x4   -0.057543
x5    0.644720
x6   -0.644719
x7    0.123658
x8   -0.031816
x9   -0.100084
dtype: float64

In [64]:
np.exp(results.params)#오즈 비 출력
# 각 값의 뜻은?

x1    0.353565
x2    0.565988
x3    0.740753
x4    0.944082
x5    1.905454
x6    0.524810
x7    1.131628
x8    0.968685
x9    0.904761
dtype: float64

In [60]:
# 모델 평가

pred_y = results.predict(X_test)
pred_y[:20]


array([0.32425336, 0.34957592, 0.86123417, 0.09718375, 0.65989377,
       0.27388715, 0.87138904, 0.70942758, 0.64525211, 0.29087617,
       0.94255607, 0.7333076 , 0.35266046, 0.14071878, 0.10209849,
       0.85648149, 0.46016374, 0.10128261, 0.27421777, 0.75816357])

In [ ]:
results2 = accuracy_score

In [ ]:
#confusion matrix

In [58]:
# ROC curve